In [1]:
'''
This file plots CMIP5 RCP - HIST over South Australia
/Users/earl/Desktop/Yang
and places the outputs in
/Users/earl/Dropbox/CMIP5/figures

Earl Duran 
created: 19-Mar-18
e.duran@unsw.edu.au
'''

import cosima_cookbook as cc
import os
import xarray as xr
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib
import sys
import pickle
import itertools
def find_nearest_index(array, value):
    return int((np.abs(array - value)).argmin())
from scipy import interpolate
from scipy import stats
from dask.distributed import Client

import cartopy.crs as ccrs
import cartopy.feature as cft
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
def arange(start,end,stride=1):
    return np.arange(start,end+0.00000001,stride)

import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata

session = cc.database.create_session()

In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
client = Client()
client

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:41452 Dashboard: http://127.0.0.1:38192/status,Cluster Workers: 4 Cores: 8 Memory: 33.67 GB


In [4]:
input_path = '/g/data/e14/erd561/NOAA_OI_SST_V2/'
output_path = '/g/data/e14/erd561/Australia_2004-2017/'

data = xr.open_dataset(input_path + 'sst.mnmean.nc')

data

<xarray.Dataset>
Dimensions:    (lat: 180, lon: 360, nbnds: 2, time: 447)
Coordinates:
  * lat        (lat) float32 89.5 88.5 87.5 86.5 ... -86.5 -87.5 -88.5 -89.5
  * lon        (lon) float32 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * time       (time) datetime64[ns] 1981-12-01 1982-01-01 ... 2019-02-01
Dimensions without coordinates: nbnds
Data variables:
    sst        (time, lat, lon) float32 ...
    time_bnds  (time, nbnds) datetime64[ns] ...
Attributes:
    title:          NOAA Optimum Interpolation (OI) SST V2
    Conventions:    CF-1.0
    history:        Wed Apr  6 13:47:45 2005: ncks -d time,0,278 SAVEs/sst.mn...
    comments:       Data described in  Reynolds, R.W., N.A. Rayner, T.M.\nSmi...
    platform:       Model
    source:         NCEP Climate Modeling Branch
    institution:    National Centers for Environmental Prediction
    References:     https://www.esrl.noaa.gov/psd/data/gridded/data.noaa.oiss...
    dataset_title:  NOAA Optimum Interpolation (OI) SST V

In [5]:
year = np.arange(2004,2018)

lonW = 88
lonE = 182

latS = -72
latN = 22

temp_z0 = xr.open_dataset(input_path + 'sst.mnmean.nc').\
                sst.\
                sel(lon=slice(lonW,lonE)).\
                sel(lat=slice(latS,latN,-1)).\
                groupby('time.year').\
                mean('time', skipna=True).\
                sel(year=slice(2004,2017))

temp_z0 = temp_z0.assign_coords(lon=temp_z0.lon-360)

mask = xr.open_dataset(input_path + 'lsmask.nc').\
                mask.\
                sel(lon=slice(lonW,lonE)).\
                sel(lat=slice(latS,latN,-1))

mask = mask.assign_coords(lon=mask.lon-360)

temp_z0 = temp_z0.where(mask==1, np.nan)

temp_z0 = temp_z0.squeeze()
print(temp_z0)

<xarray.DataArray 'sst' (year: 14, lat: 94, lon: 94)>
array([[[       nan,        nan,        nan, ..., -1.3566667,
         -1.3491665, -1.3491665],
        [       nan,        nan,        nan, ..., -1.3100001,
         -1.2958333, -1.2925   ],
        [       nan,        nan,        nan, ..., -1.2375001,
         -1.2241668, -1.2233334],
        ...,
        [27.764162 , 27.723335 , 27.684168 , ..., 27.169165 ,
         27.119165 , 27.065002 ],
        [27.537498 , 27.47917  , 27.413336 , ..., 26.963331 ,
         26.923334 , 26.871666 ],
        [27.313334 , 27.218332 , 27.030832 , ..., 26.704168 ,
         26.677498 , 26.635002 ]],

       [[       nan,        nan,        nan, ..., -1.1725   ,
         -1.2058333, -1.2333333],
        [       nan,        nan,        nan, ..., -1.1833333,
         -1.2258333, -1.2591666],
        [       nan,        nan,        nan, ..., -1.2016666,
         -1.2441666, -1.2741667],
        ...,
        [28.002497 , 28.031664 , 28.088333 , ..., 27.2

In [6]:
lat = temp_z0.lat
lon = temp_z0.lon

# temp_z0_trans = np.transpose(temp_z0, (2,1,0))
temp_z0_trans = temp_z0
print(np.shape(temp_z0_trans))
temp_z0_slope = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='temp_z0')
temp_z0_p_value = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='temp_z0')
temp_z0_std_err = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='temp_z0')
for iid, i in enumerate(lat):
    for jid, j in enumerate(lon):
        temp_z0_slope[jid, iid], _, _, temp_z0_p_value[jid, iid], temp_z0_std_err[jid, iid] = \
        stats.linregress(year, temp_z0_trans[:, jid, iid])
    print('lat ' + str(np.array(i)))
print(temp_z0_slope)

(14, 94, 94)
lat -71.5
lat -70.5
lat -69.5
lat -68.5
lat -67.5
lat -66.5
lat -65.5
lat -64.5
lat -63.5
lat -62.5
lat -61.5
lat -60.5
lat -59.5
lat -58.5
lat -57.5
lat -56.5
lat -55.5
lat -54.5
lat -53.5
lat -52.5
lat -51.5
lat -50.5
lat -49.5
lat -48.5
lat -47.5
lat -46.5
lat -45.5
lat -44.5
lat -43.5
lat -42.5
lat -41.5
lat -40.5
lat -39.5
lat -38.5
lat -37.5
lat -36.5
lat -35.5
lat -34.5
lat -33.5
lat -32.5
lat -31.5
lat -30.5
lat -29.5
lat -28.5
lat -27.5
lat -26.5
lat -25.5
lat -24.5
lat -23.5
lat -22.5
lat -21.5
lat -20.5
lat -19.5
lat -18.5
lat -17.5
lat -16.5
lat -15.5
lat -14.5
lat -13.5
lat -12.5
lat -11.5
lat -10.5
lat -9.5
lat -8.5
lat -7.5
lat -6.5
lat -5.5
lat -4.5
lat -3.5
lat -2.5
lat -1.5
lat -0.5
lat 0.5
lat 1.5
lat 2.5
lat 3.5
lat 4.5
lat 5.5
lat 6.5
lat 7.5
lat 8.5
lat 9.5
lat 10.5
lat 11.5
lat 12.5
lat 13.5
lat 14.5
lat 15.5
lat 16.5
lat 17.5
lat 18.5
lat 19.5
lat 20.5
lat 21.5
<xarray.DataArray 'temp_z0' (lat: 94, lon: 94)>
array([[        nan,         nan,        

In [7]:
temp_z0_xr = xr.DataArray(np.transpose(temp_z0.values, (1,2,0)), name='temp_z0_oisst',
                        coords=[lat, lon, year], 
                        dims=['lat', 'lon', 'year'])
print(temp_z0_xr)
temp_z0_xr.to_netcdf(output_path + 'temp_z0_oisst.nc')

temp_z0_slope_xr = xr.DataArray(temp_z0_slope, name='temp_z0_slope_oisst',
                        coords=[lat, lon], 
                        dims=['lat', 'lon'])
print(temp_z0_slope_xr)
temp_z0_slope_xr.to_netcdf(output_path + 'temp_z0_slope_oisst.nc')

temp_z0_p_value_xr = xr.DataArray(temp_z0_p_value, name='temp_z0_p_value_oisst',
                        coords=[lat, lon], 
                        dims=['lat', 'lon'])
print(temp_z0_p_value_xr)
temp_z0_p_value_xr.to_netcdf(output_path + 'temp_z0_p_value_oisst.nc')

temp_z0_std_err_xr = xr.DataArray(temp_z0_std_err, name='temp_z0_std_err_oisst',
                        coords=[lat, lon], 
                        dims=['lat', 'lon'])
print(temp_z0_std_err_xr)
temp_z0_std_err_xr.to_netcdf(output_path + 'temp_z0_std_err_oisst.nc')

<xarray.DataArray 'temp_z0_oisst' (lat: 94, lon: 94, year: 14)>
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [-1.3566667, -1.1725   , -1.3258333, ..., -1.4874998,
         -1.4291667, -1.3766667],
        [-1.3491665, -1.2058333, -1.2708333, ..., -1.4833332,
         -1.4408334, -1.4025   ],
        [-1.3491665, -1.2333333, -1.2591667, ..., -1.4833335,
         -1.4583334, -1.4016666]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [-1.3100001, -1.1833333, -1.3666664,

In [5]:
# input_path = '/g/data/e14/erd561/IMOS/'
# output_path = '/g/data/e14/erd561/Australia_3/'

# data = xr.open_dataset(input_path + 'IMOS_aggregation_SSTAARS_20181023T031550Z.nc')

# data

<xarray.Dataset>
Dimensions:                  (lat: 2000, lon: 3500, time: 313)
Coordinates:
  * lat                      (lat) float32 -20.01 -20.03 ... -59.97 -59.99
  * lon                      (lon) float32 100.01 100.03 ... 169.97 169.99
  * time                     (time) datetime64[ns] 1992-03-16T15:20:00 ... 2018-03-16T15:20:00
Data variables:
    sea_surface_temperature  (time, lat, lon) float32 ...
    sst_dtime                (time, lat, lon) float32 ...
    dt_analysis              (time, lat, lon) float32 ...
    satellite_zenith_angle   (time, lat, lon) float32 ...
    l2p_flags                (time, lat, lon) float32 ...
    quality_level            (time, lat, lon) float64 ...
    sses_bias                (time, lat, lon) float32 ...
    sses_standard_deviation  (time, lat, lon) float32 ...
    sses_count               (time, lat, lon) float32 ...
Attributes:
    Conventions:                pseudo CF-1.4?
    title:                      IMOS L3S Nighttime gridded multip

In [ ]:
# temp_z0_raw1 = xr.open_dataset(input_path + 'IMOS_aggregation_SSTAARS_20181023T031550Z.nc').\
#                 sea_surface_temperature.\
#                 interp(lon=arange(100,170,0.1)).\
#                 interp(lat=arange(-60,-20,0.1))
# print(temp_z0_raw1)

# temp_z0_raw2 = temp_z0_raw1.\
#                 groupby('time.year').\
#                 mean('time', skipna=True)
# print(temp_z0_raw2)

# temp_z0_raw3 = temp_z0_raw2.\
#                 sel(year=slice(2004,2017))
# print(temp_z0_raw3)


# temp_z0 = temp_z0.assign_coords(lon=temp_z0.lon-360)
# print(temp_z0)

In [ ]:
# lat = temp_z0.lat
# lon = temp_z0.lon

# # temp_z0_trans = np.transpose(temp_z0, (2,1,0))
# temp_z0_trans = temp_z0
# print(np.shape(temp_z0_trans))
# temp_z0_slope = xr.DataArray(
#     np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
#     'lat', 'lon'), coords=[lat, lon], name='temp_z0')
# temp_z0_p_value = xr.DataArray(
#     np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
#     'lat', 'lon'), coords=[lat, lon], name='temp_z0')
# temp_z0_std_err = xr.DataArray(
#     np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
#     'lat', 'lon'), coords=[lat, lon], name='temp_z0')
# for iid, i in enumerate(lat):
#     for jid, j in enumerate(lon):
#         temp_z0_slope[jid, iid], _, _, temp_z0_p_value[jid, iid], temp_z0_std_err[jid, iid] = \
#         stats.linregress(year, temp_z0_trans[:, jid, iid])
#     print('lat ' + str(np.array(i)))
# print(temp_z0_slope)

In [ ]:
# temp_z0_xr = xr.DataArray(np.transpose(temp_z0.values, (1,2,0)), name='temp_z0_oisst',
#                         coords=[lat, lon, year], 
#                         dims=['lat', 'lon', 'year'])
# print(temp_z0_xr)
# temp_z0_xr.to_netcdf(output_path + 'temp_z0_oisst.nc')

# temp_z0_slope_xr = xr.DataArray(temp_z0_slope, name='temp_z0_slope_oisst',
#                         coords=[lat, lon], 
#                         dims=['lat', 'lon'])
# print(temp_z0_slope_xr)
# temp_z0_slope_xr.to_netcdf(output_path + 'temp_z0_slope_oisst.nc')

# temp_z0_p_value_xr = xr.DataArray(temp_z0_p_value, name='temp_z0_p_value_oisst',
#                         coords=[lat, lon], 
#                         dims=['lat', 'lon'])
# print(temp_z0_p_value_xr)
# temp_z0_p_value_xr.to_netcdf(output_path + 'temp_z0_p_value_oisst.nc')

# temp_z0_std_err_xr = xr.DataArray(temp_z0_std_err, name='temp_z0_std_err_oisst',
#                         coords=[lat, lon], 
#                         dims=['lat', 'lon'])
# print(temp_z0_std_err_xr)
# temp_z0_std_err_xr.to_netcdf(output_path + 'temp_z0_std_err_oisst.nc')